# FAIR and scalable management of small-angle X-ray scattering data  
## Module 3: Data analysis and visualization
### 3.2: Analysis

> Authors: Torsten Giess, Selina Itzigehl  
> Last modified: 15.12.2022

---

### **Abstract** <a class="anchor" name="abstract"></a>

Using novel package [sastools](https://github.com/FAIRChemistry/SAS-tools) (version 0.3.2), as well as established packages numpy (version 1.23.5) and pandas (version 1.5.2), as well as packages from the Python 3 standard libary, this notebook provides the means to determine the lyotropic liquid crystalline phase and lattice parameter _a_ from the peak centers of a diffractogram by calibration of scattering vectors and calculation of lattice plane distances. 

---

### **Table of Contents** <a class="anchor" name="table_of_contents"></a>

- [Abstract](#abstract)
- [Workflow](#workflow)
    - [User guide](#user_guide)
    - [Preparation](#preparation)
    - [Analysis with Python](#analysis)
- [Disclosure](#disclosure)

---

### **Workflow** <a class="anchor" name="workflow"></a>

Following is the workflow for Module 3.2: Analysis of FAIR and scalable management of small-angle X-ray scattering data.

#### **User guide** <a class="anchor" name="user_guide"></a>

This notebook can be used to determine the lyotropic liquid crystalline phase as well as the corresponding lattice parameter _a_ from the peak centers obtained from previous evaluation of diffractograms by providing calibration and calculation of required values.

#### **Preparation** <a class="anchor" name="preparation"></a>

This section contains the necessary preparations for using this module. Code cells in this section are required regardless of which functionality of this notebook is used. First, the required packages from the [Python 3 standard library](https://docs.python.org/3/library/), the Python Package Index ([PyPI](https://pypi.org/)), and *ad hoc* modules of this work are imported. Then, both current time and path are retrieved and stored in the desired formats.

In [32]:
print("Importing standard library packages.")
from datetime import date
from pathlib import Path
from typing import List
print ("Done.")

Importing standard library packages.
Done.


In [33]:
print("Importing PyPI packages.")
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyaniml import AnIMLDocument, Category, Parameter
from sastools.analyzer import SAXSStandards, LLCPhases, PrepareStandard, LLCAnalyzer
from sastools.readers import OriginReader, infer_type
from sklearn.metrics import r2_score
print ("Done.")

Importing PyPI packages.
Done.


In [34]:
date_suffix = str(date.today()).replace("-", "")[2:]

In [35]:
cwd = Path.cwd()
path_to_datasets = cwd / "./datasets/"

---

#### **Analysis with Python** <a class="anchor" name="analysis"></a>

Determine the lyotropic liquid crystalline (LLC) phase and calculate the lattice parameter from measured datasets and store the results in the respective AnIML file(s).

Open AnIML document to later append analysis results from this Notebook.

In [36]:
path_to_AnIML_file = path_to_datasets / f"processed/fairsaxs_220512.animl"

In [37]:
with path_to_AnIML_file.open("r") as f:
    xml_string = f.read()
    animl_doc = AnIMLDocument.fromXMLString(xml_string)

Access the Result of the ExperimentStep this analysis belongs to.

In [38]:
for index, item in enumerate(animl_doc.experiment_step_set.experiment_steps):
    print(index, f"ID: {item.experiment_step_id};    Name: {item.name}")

0 ID: CholPal_cal_20220214;    Name: Calibration data
1 ID: OTAB_010wtp_T025;    Name: Sample data for OTAB/water: x = 010 wt%; T = 25 C
2 ID: OTAB_020wtp_T025;    Name: Sample data for OTAB/water: x = 020 wt%; T = 25 C
3 ID: OTAB_030wtp_T025;    Name: Sample data for OTAB/water: x = 030 wt%; T = 25 C
4 ID: OTAB_040wtp_T025;    Name: Sample data for OTAB/water: x = 040 wt%; T = 25 C
5 ID: OTAB_050wtp_T025;    Name: Sample data for OTAB/water: x = 050 wt%; T = 25 C
6 ID: OTAB_061wtp_T025;    Name: Sample data for OTAB/water: x = 061 wt%; T = 25 C
7 ID: OTAB_062wtp_T025;    Name: Sample data for OTAB/water: x = 062 wt%; T = 25 C
8 ID: OTAB_063wtp_T025;    Name: Sample data for OTAB/water: x = 063 wt%; T = 25 C
9 ID: OTAB_064wtp_T025;    Name: Sample data for OTAB/water: x = 064 wt%; T = 25 C
10 ID: OTAB_065wtp_T025;    Name: Sample data for OTAB/water: x = 065 wt%; T = 25 C
11 ID: OTAB_066wtp_T025;    Name: Sample data for OTAB/water: x = 066 wt%; T = 25 C
12 ID: OTAB_067wtp_T025;    Nam

In [39]:
experiment_step = animl_doc.experiment_step_set.experiment_steps[60].result

Create the new Category for the analysis results.

In [40]:
new_category = Category(name="Analyses")

Import Lorentzian fit data from Origin TXT output file. For several measurement datasets select the respective folder.

In [41]:
available_txt_files = [file for file in (path_to_datasets / "raw/OTAC_measurement_data/Lorentzian_fitting_data").glob("*.txt")]
print([file.name for file in available_txt_files])

['CholPal_20220214_lorentz.txt', 'OTAC_070wtp_T025_lorentz.txt', 'OTAC_090wtp_T010_lorentz.txt', 'OTAC_094wtp_T050_lorentz.txt', 'OTAC_094wtp_T068_lorentz.txt']


In [42]:
dict_of_df = {}
i=0
for file in available_txt_files:
    dict_of_df[available_txt_files[i].name] = OriginReader(available_txt_files[i]).xc_dataframe
    i += 1
print(dict_of_df.keys())

dict_keys(['CholPal_20220214_lorentz.txt', 'OTAC_070wtp_T025_lorentz.txt', 'OTAC_090wtp_T010_lorentz.txt', 'OTAC_094wtp_T050_lorentz.txt', 'OTAC_094wtp_T068_lorentz.txt'])


Calculate the scattering vectors for calibration from literature lattice plane distances (given in Angstrom).

In [43]:
prepare_standard = PrepareStandard(SAXSStandards.CHOLESTERYL_PALMITATE)
q_cholpal_literature = prepare_standard.q_std_lit
print(q_cholpal_literature)

[1.1968372019465192, 2.3936744043489915, 3.590511605155628]


Calibrate the peak centers of a measurement (element from `available_txt_files`) with the calibration line. Then calculate the lattice plane ratio from `q_corrected`:

In [44]:
slope_and_intercept = prepare_standard.calculate_linear_regression(
    q_std_meas=dict_of_df[available_txt_files[0].name]["value"].tolist()
)
print(slope_and_intercept)

(1.0179868569279822, 0.008622816851908443)


In [45]:
llc_analyzer = LLCAnalyzer()
llc_analyzer.calibrate_data(
    slope=slope_and_intercept[0],
    q_meas=dict_of_df[available_txt_files[0].name]["value"].tolist(),
    intercept=slope_and_intercept[1]
)
q_corrected = llc_analyzer.q_corr
print(q_corrected)

[1.1965320399383095, 2.394285195931204, 3.590205975581628]


Add corrected $q$ values to the AnIML document:

In [46]:
subcategory = Category(name="q_corrected")

In [47]:
for i, q in enumerate(q_corrected):
    new_parameter = Parameter(
        name=f"q_corrected of peak {i+1}",
        parameter_type=infer_type(q),
        value=q
    )
    subcategory.add_content(new_parameter)

In [48]:
new_category.add_content(subcategory)

Calculate lattice plance distances $d$ and their ratios from the corrected $q$ values:

In [49]:
llc_analyzer.calculate_lattice_ratio()
d_measured = llc_analyzer.d_measured
print(d_measured)
d_ratio = llc_analyzer.d_ratio
print(d_ratio)

[5.251163443566069, 2.624242641543745, 1.7500904822491932]
[0.4997449936088106, 0.33327671116264196]


Add the measured $d$ values to the AnIML document:

In [50]:
subcategory = Category(name="d_measured")

In [51]:
for i, d in enumerate(d_measured):
    new_parameter = Parameter(
        name=f"d_measured of peak {i+1}",
        parameter_type=infer_type(d),
        value=d
    )
    subcategory.add_content(new_parameter)

In [52]:
new_category.add_content(subcategory)

To determine the lyotropic liquid crystalline phase at hand, calculate the _d_ ratios. With the _d_ ratio(s), the phase and respective lattice parameter _a_ are determined:

Add `d_ratio` to the AnIML document:

In [53]:
subcategory = Category(name="d_ratio")

In [54]:
for i, d in enumerate(d_ratio):
    new_parameter = Parameter(
        name=f"d_ratio of peaks {i+2} and 1",
        parameter_type=infer_type(d),
        value=d
    )
    subcategory.add_content(new_parameter)

In [55]:
new_category.add_content(subcategory)

Determine the LLC phase from the lattice plane ratio and calculate the respective lattice parameter $a$:

In [56]:
phase = llc_analyzer.determine_phase()
print(phase)

Lamellar LLC Phase


In [57]:
phase.calculate_lattice_parameters(d_meas=d_measured)
phase_information = phase.phase_information
print(phase_information)

{'phase': 'La', 'lattice_parameter': 5.251163443566069}


Add the phase information to the AnIML document:

In [58]:
subcategory = Category(name="phase information")

In [59]:
new_parameter = Parameter(
    name=f"phase",
    parameter_type=infer_type(phase_information["phase"]),
    value=phase_information["phase"]
)
subcategory.add_content(new_parameter)

In [60]:
new_parameter = Parameter(
    name=f"lattice parameter",
    parameter_type=infer_type(phase_information["lattice_parameter"]),
    value=phase_information["lattice_parameter"]
)
subcategory.add_content(new_parameter)

In [61]:
new_category.add_content(subcategory)

If the phase is not cubic, add everything to the existing AnIML document and serialize it (else, skip to Cubic phase specification):

In [62]:
experiment_step.add_result(new_category)

In [63]:
xml_string = animl_doc.toXML()

In [64]:
with path_to_AnIML_file.open("w") as f:
    f.write(xml_string)
del animl_doc, xml_string
print(f"Successfully created AnIML document.")

Successfully created AnIML document.


---

#### **Cubic phase specification** <a class="anchor" name="phases"></a>

As the multiple scattering maxima of the cubic LLC phase can be assigned to various Miller indices, different space groups are possible. This section enables checking for the best fit by plotting experimentally determined values for $d^{-1}$ vs. $\sqrt{h^2+k^2+l^2}$.

In [65]:
data = pd.read_table(path_to_datasets / "raw/OTAC_measurement_data/Lorentzian_fitting_data/OTAC_094wtp_T050_lorentz.txt",
                     delimiter="\t",
                     usecols=(1,2),
                     names=("key", "value"),
                     header=1,
                     engine="python"
                    )

peak_centers = data[data["key"] == "xc"]
scattering_vector = peak_centers["value"].values.tolist()

phase.calculate_sqrt_miller()
x = phase.sqrt_miller
phase.calculate_d_reciprocal(peak_center=scattering_vector)
y = phase.d_reciprocal
print(x)
print(y)

AttributeError: 'LamellarPhase' object has no attribute 'calculate_sqrt_miller'

The calculated values for $d^{-1}$ plotted against the respective $\sqrt{h^2+k^2+l^2}$ and fitted by linear regression. The $R^2$-value is calculated for assessment of the fit quality:

In [ ]:
coef = np.polyfit(x, y, 1)
fit = np.poly1d(coef)
r2 = r2_score(y, fit(x))

For visualization, the fit is plotted:

In [ ]:
fig = plt.figure(dpi=600)
ax = fig.add_subplot()
ax.set(xlabel="$\sqrt{h^2+k^2+l^2}$",
       ylabel="$d^{-1}$ / nm"
       )
ax.plot(x, y, marker=".", linestyle="None", color="black")
ax.plot(x, fit(x), marker="None", linestyle=":", color="black")

for i, label in enumerate(h):
    ax.annotate(("("+str(h[i])+" "+str(k[i])+" "+str(l[i])+")"), (x[i], y[i]))
    
ax.text(3.0, 0.39, f"$y$ = {str(fit)}")
ax.text(3.0, 0.35, f"$R^2$ = {str(round(r2, 4))}")

plt.show()

Optionally, add exact cubic phase to the AnIML document and serialize it:

In [ ]:
exact_cubic_phase = LLCPhases.INDETERMINATE
phase.exact_phase = exact_cubic_phase

In [ ]:
for category in experiment_step.results[1].content:
    if category.name == "phase information":
        new_parameter = Parameter(
            name=f"exact cubic phase",
            parameter_type=infer_type(phase.exact_phase.value),
            value=phase.exact_phase.value
        )
        category.add_content(new_parameter)
    else:
        pass

In [ ]:
xml_string = animl_doc.toXML()

In [ ]:
with path_to_AnIML_file.open("w") as f:
    f.write(xml_string)
del animl_doc, xml_string
print(f"Successfully created AnIML document.")

---

### **Disclosure** <a class="anchor" name="disclosure"></a>

**Contributions**

If you wish to contribute to the FAIR Chemistry project, find us on [GitHub](https://github.com/FAIRChemistry)!

**MIT License**

Copyright (c) 2022 FAIR Chemistry

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.